<a href="https://colab.research.google.com/github/KamonohashiPerry/MachineLearning/blob/master/Causal_Inference/Python_Causal_Inference_Chap8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## SAM(Structural Agnostic Model)
+ 識別器Dのネットワーク構造
 + バッチノーマライゼーション
+ 生成器G
 + 1次元のバッチノーマライゼーション
  + 複数個の変数の値を一気に生成するのではなく、ある一つを生成し、残りは観測データを与えて、一つだけ生成するようにする。（ギブスサンプリングみたいな感じ）

In [1]:
# PyTorchのバージョンを下げる
!pip install torch==1.4.0+cu92 torchvision==0.5.0+cu92 -f https://download.pytorch.org/whl/torch_stable.html

Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |████████████████████████████████| 640.6MB 22kB/s 
     |████████████████████████████████| 4.0MB 18.3MB/s 
  Found existing installation: torch 1.6.0+cu101
    Uninstalling torch-1.6.0+cu101:
      Successfully uninstalled torch-1.6.0+cu101
  Found existing installation: torchvision 0.7.0+cu101
    Uninstalling torchvision-0.7.0+cu101:
      Successfully uninstalled torchvision-0.7.0+cu101


In [2]:
import torch
print(torch.__version__)

1.4.0+cu92


In [3]:
!pip install cdt==0.5.18

     |████████████████████████████████| 921kB 3.3MB/s 
  Created wheel for GPUtil: filename=GPUtil-1.4.0-cp36-none-any.whl size=7413 sha256=c771880ce4a5dca99f647cc20196d248e728b48083df90fc1ad8e4464e17d05f
  Stored in directory: /root/.cache/pip/wheels/3d/77/07/80562de4bb0786e5ea186911a2c831fdd0018bda69beab71fd
  Created wheel for skrebate: filename=skrebate-0.61-cp36-none-any.whl size=29257 sha256=7d83aa5edc91eee2642e08d5819ce211730d538d48914a0ac3f970f1c70ae75e
  Stored in directory: /root/.cache/pip/wheels/ae/d8/ae/9b51d487e9d02219996d6c260255a216ef07d905b0a0b00ce3
Successfully built GPUtil skrebate


In [4]:
# 乱数のシードを設定
import random
import numpy as np

np.random.seed(1234)
random.seed(1234)

In [6]:
# 使用するパッケージ（ライブラリと関数）を定義
# 標準正規分布の生成用
from numpy.random import *

# グラフの描画用
import matplotlib.pyplot as plt

# その他
import pandas as pd

# シグモイド関数をimport
from scipy.special import expit

# データ数
num_data = 2000

# 部下育成への熱心さ
x = np.random.uniform(low=-1, high=1, size=num_data)  # -1から1の一様乱数

# 上司が「上司向け：部下とのキャリア面談のポイント研修」に参加したかどうか
e_z = randn(num_data)  # ノイズの生成
z_prob = expit(-5.0*x+5*e_z)
Z = np.array([])

# 上司が「上司向け：部下とのキャリア面談のポイント研修」に参加したかどうか
for i in range(num_data):
    Z_i = np.random.choice(2, size=1, p=[1-z_prob[i], z_prob[i]])[0]
    Z = np.append(Z, Z_i)

# 介入効果の非線形性：部下育成の熱心さxの値に応じて段階的に変化
t = np.zeros(num_data)
for i in range(num_data):
    if x[i] < 0:
        t[i] = 0.5
    elif x[i] >= 0 and x[i] < 0.5:
        t[i] = 0.7
    elif x[i] >= 0.5:
        t[i] = 1.0

e_y = randn(num_data)
Y = 2.0 + t*Z + 0.3*x + 0.1*e_y 


# 本章からの追加データを生成

# Y2：部下当人のチームメンバへの満足度 1から5の5段階
Y2 = np.random.choice([1.0, 2.0, 3.0, 4.0, 5.0],
                      num_data, p=[0.1, 0.2, 0.3, 0.2, 0.2])

# Y3：部下当人の仕事への満足度
e_y3 = randn(num_data)
Y3 = 3*Y + Y2 + e_y3

# Y4：部下当人の仕事のパフォーマンス
e_y4 = randn(num_data)
Y4 = 3*Y3 + 2*e_y4 + 5

df = pd.DataFrame({'x': x,
                   'Z': Z,
                   't': t,
                   'Y': Y,
                   'Y2': Y2,
                   'Y3': Y3,
                   'Y4': Y4,
                   })

del df["t"]  # 変数tは観測できないので削除

df.head()  # 先頭を表示

,x,Z,Y,Y2,Y3,Y4
0,-0.807894,1.0,2.291971,3.0,10.087388,33.942040
1,0.819267,0.0,2.169256,2.0,9.693777,36.705939
2,-0.907815,1.0,2.247729,4.0,9.710220,33.767494
3,-0.033905,1.0,2.543262,1.0,9.692669,36.801544
4,-0.060081,0.0,1.924453,4.0,10.077395,33.666739


### 識別器Dの実装

In [7]:
# PyTorchから使用するものをimport
import torch
import torch.nn as nn

class SAMDiscriminator(nn.Module):
  def __init__(self, nfeatures, dnh, hlayers):
    super.nfeatures = nfeatures # 入力変数の数
    layers = []
    layers.append(nn.Linear(nfeatures, dnh))
    layers.append(nn.BatchNorm1d(dnh))
    layers.append(nn.LeakyReLU(.2))

    for i in range(hlayers - 1):
      layers.append(nn.Linear(dnh, dnh))
      layers.append(nn.BatchNorm1d(dnh))
      layers.append(nn.LeakyReLU(.2))

    layers.append(nn.Linear(dnh, 1))

    self.layers = nn.Sequential(*layers)

    # mask
    mask = torch.eye(nfeatures, nfeatures)
    self.register_buffer('mask', mask.unsqueeze(0))

  def forward(self, input, obs_data=None):
    # 順伝播の計算
    if obs_data is not None:
      # 生成データを識別器に入力する場合
      return [self.layesr(i) for i in torch.unbind(obs_data.unsqueeze(1) * (1-self.mask) + input.unsqueeze(1)*self.mask, 1) ]

    else:
      # 観測データを識別器に入力する場合
      return self.layers(input)

  def reset_parameters(self):
    # 識別器Dの重みパラメータの初期化
    for layer in self.layers:
      if hasattr(layer, 'reset_parameters'):
        layer.reset_parameters()

### 生成器Gの実装

In [10]:
from cdt.utils.torch import ChannelBatchNorm1d, MatrixSampler, Linear3D

class SAMGenerator(nn.Module):
  def __init__(self, data_shape, nh):
    # 初期化
    super(SAMGenerator, self).__init__()

    nb_vars = data_shape[1]
    skeleton = 1 - torch.eye(nb_vars + 1, nb_vars)

    self.register_buffer('skeleton', skeleton)

    # ネットワークの用意
    ## 入力層
    self.input_layer = Linear3D((nb_vars, nb_vars + 1, nh))

    ## 中間層
    layers = []
    # 2次元を1次元に変換してバッチノーマライゼーションする
    layers.append(ChannelBatchNorm1d(nb_vars, nh))
    layers.append(nn.Tanh())
    self.layers = nn.Sequential(*layers)

    ## 出力層
    self.output_layer = Linear3D((nb_vars, nh, 1))

  def forward(self, data, noise, adj_matrix, drawn_neurons=None):
    # 順伝播の計算
    ## 入力層
    x = self.input_layer(data, noise, adj_matrix * self.skeleton)

    ## 中間層
    x = self.layers(x)

    ## 出力層
    output = self.output_layer(x, noise=None, adj_matrix=drawn_neurons)

    return output.squeeze(2)

  def reset_parameters(self):
    # 重みパラメータの初期化を実施
    self.input_layer.reset_parameters()
    self.output_layer.reset_parameters()

    for layer in self.layers:
      if hasattr(layer, 'reset_parameters'):
        layer.reset_parameters()

## SAMの損失関数
+ DAGを生み出す損失関数

In [11]:
# ネットワークを示す因果構造マトリクスMがDAGになるように加える損失
def notears_constr(adj_m, max_pow=None):
  m_exp = [adj_m]
  if max_pow is None:
    max_pow = adj_m.shape[1]
  while(m_exp[-1].sum() > 0 and len(m_exp) < max_pow ):
    m_exp.append(m_exp[-1] @ adj_m/len(m_exp))

  return sum([i.diag().sum() for idx, i in enumerate(m_exp)])

### SAMの学習の実行

In [12]:
from sklearn.preprocessing import scale
from torch import optim
from torch.utils.data import DataLoader
from tqdm import tqdm

def run_SAM(in_data, lr_gen, lr_disc, lambda1, lambda2, hlayers, nh, dnh, train_epochs, test_epochs, device):
  # SAMの学習の実行
  ## 入力データの前処理
  list_nodes = list(in_data.columns)
  data = scale(in_data[list_nodes].values)
  nb_var = len(list_nodes)
  data = data.astype('float32')
  data = torch.from_numpy(data).to(device)
  rows, cols = data.size()

  # DataLoader
  batch_size = rows
  data_iterator = DataLoader(data, batch_size=batch_size,
                              shuffle=True, drop_last=True)
  
  # ネットワークの生成とパラメータの初期化
  sam = SAMGenerator((batch_size, cols), nh).to(device)
  graph_sampler = MatrixSampler(nb_var, mask=None, gumber=False).to(device)
  neuron_sampler = MatrixSampler((nh, nb_var), mask=False, gumbel=True)

  # 重みパラメータの初期化
  sam.reset_parameters()
  graph_sampler.weights.data.fill_(2)

  # ネットワークの生成とパラメータの初期化
  discriminator = SAMDiscriminator(cols, dnh, hlayerrs).to(device)
  discriminator.reset_parameters()

  # 最適化の設定
  ## 生成器
  g_optimizer = optim.Adam(sam.parameters(), lr=lr_gen)
  graph_optimizer = optim.Adam(graph_sampler.parameters(), lr=lr_gen)
  neuron_optimizer = optim.Adam(neuron_sampler.parameters(), lr=lr_gen)

  ## 識別器
  d_optimizer = optim.Adam(discriminator.parameters(), lr=lr_disc)

  ## 損失関数
  criterion = nn.BCEWithLogitsLoss()

  ## 損失関数のDAGに関する制約の設定パラメータ
  dagstart = 0.5
  dagpenalization_increase = 0.001*10

  ## forward計算、損失関数の計算に使用する変数
  _true = torch.ones(1).to(device)
  _false = torch.zeros(1).to(device)

  noise = torch.randn(batch_size, nb_var).to(device)
  noise_row = torch.ones(1, nb_var).to(device)

  output = torch.zeros(nb_var, nb_var).to(device)
  output_loss = torch.zeros(1, 1).to(device)
  
  ## forward計算、ネットワーク学習
  pbar = tqdm(range(train_epochs + test_epochs))

  for epoch in pbar:
    for i_batch, batch in enumerate(data_iterator):
      # 最適化を初期化
      g_optimizer.zero_grad()
      graph_optimizer.zero_grad()
      neuron_optimizer.zero_grad()
      d_optimizer.zero_grad()

      # 因果構造マトリクスと複雑さマトリクスの取得
      drawn_graph = graph_samplet()
      drawn_neurons = neuron_sampler()

      # ノイズをリセットし、生成器Gで擬似データを生成
      noise.normal_()
      generated_variables = sam(data=batch, noise=noise,
                                  adj_matrix=torch.cat([drawn_graph, noise_row], 0), drawn_neurons=drawn_neurons)
      
      # 識別器Dで判定
      disc_vars_d = discriminator(generated_variables.detach(), batch)
      # 観測変数のリスト
      disc_vars_g = discriminator(generated_variables, batch)
      true_vars_disc = discriminator(batch)

      # 損失関数の計算
      disc_loss = sum([criterion(gen, _false.expand_as(gen)) for gen in disc_vars_d ])/ nb_var + criterion(true_vars_disc, _true.expand_as(true_vars_disc))

      gen_loss = sum([criterion(gen, _true.expand_as(gen)) for gen in disc_vars_g ])

      # 識別器Dのバックプロパゲーションとパラメータの更新
      if epoch < train_epochs:
        disc_loss.backward()
        d_optimizer.step()

      # 生成器のGの損失の計算の残り
      struc_loss = lambda1 / batch_size*drawn_graph.sum()
      func_loss = lambda2 / batch_size*drawn_neurons.sum()

      regul_loss = gen_loss + func_loss

      if epoch <= train_epochs * dagstart:
        # epochが基準前のときはDAGになるようにMへのNo Tearsの制限はかけない
        loss = gen_loss + regul_loss

      else:
        # epochが基準後のときはDAGになるようにMへのNo Tearsの制限をかける
        filters = graph_sampler.get_proba()
        # マトリクスMの要素を取得
        dag_constraint = notears_constr(filters*filters)

        # 徐々に線形にDAFの正則を強くする
        loss = gen_loss + regul_loss + ((epoch-train_epochs*dagstart)*dagpenalization_increase)*dag_constraint

      if epoch >= train_epochs:
        # testのepochの場合、結果を取得
        output.add_(filters.data)
        output_loss.add_(gen_loss.data)

      else:
        # trainのepochの場合、生成器Gのバックプロパゲーションと更新
        loss.backward(retain_graph=True)
        g_optimizer.step()
        graph_optimizer.step()
        neuron_optimizer.step()

      # 進捗の表示
      if epoch % 50 ==0:
        pbar.set_postfix(gen=gen_loss.item()/cols, disc=disc_loss.item(),
                         regul_loss=regul_loss.item(),
                         tot=loss.item())
        
  return output.cpu().numpy()/test_epochs, output_loss.cpu().numpy()/test_epochs/cols